# TSV Maker for housing characteristics using PUMA data

**Contributors** : Tobi Adekanye, Anthony Fontanini

**Date Created** : January 14, 2020

Housing characteristics tsvs are presently created using EPW weather files. However, these are large areas with multiple cities and counties making up an EPW region. We use Public Use Microdata Areas (PUMA) to create smaller, more spatially relevant regions to represent some of the housing characteristics. This notebook restructures the PUMA data to create the relevant tsvs housing characteristics. 

There are two files that need to be downloaded for the tsv maker to work:

1. **PUMA data (csv)** : this file contains the different housing characteristics and household weights for the PUMA areas
2. **Data dictionary (xls)** : the PUMA data codes the different housing characteristics as integer data. This file   contains several sheets which map the integer data to the respective names. It also contains ResStock mapping in the case that the census mapping does not match that which presently is used in-house. 

# Inputs

Some inputs for this notebook.
- `write_tf(bool)`: Flag to write the created tsv files
- `project(string)`: the name of the project file where the tsv will be copied into. This input does not matter if `write_tf=False`.
- `dep_lst(list(str))`: list of the dependencies needed for the tsv file
- `option_col(string)`: the name of the field which the dependencies will be distributed against

**Note**: dep_lst and option_col need to be specified as named in the PUMA tsv file else the tsv maker will not run as appropriate. 

# PUMA fields list

The columns contained in the PUMA data csv file are as follows:

- **YEAR**: Census Year
- **MULTYEAR**: Actual year of survey, multiyear ACS
- **SAMPLE**: IPUMS sample identifier
- **SERIAL**: Household serial number
- **CBSERIAL**: Original Census Bureau household serial number
- **HHWT**: Household weight
- **STATEICP**: State (ICPSR code)
- **STATEFIP**: State (FIPS code)
- **COUNTYICP**: County (ICPSR code)
- **COUNTYFIP**: County (FIPS code)
- **PUMA**: Public Use Microdata Area
- **GQ**: Group quarters status
- **OWNERSHP**: Ownership of dwelling (tenure) (general version)
- **OWNERSHPD**: Ownership of dwelling (tenure) (detailed version)
- **COSTELEC**: Annual electricity cost
- **COSTGAS**: Annual gas cost
- **COSTWATER**: Annual water cost
- **COSTFUEL**: Annual home heating fuel
- **VACANCY**: Vacancy status
- **ROOMS**: Number of rooms
- **BUILTYR2**: Age of structure (decade)
- **UNITSSTR**: Units in structure
- **BEDROOMS**: Number of bedrooms
- **FUELHEAT**: Home heating fuel
- **PERNUM**: Person number in sample unit
- **PERWT**: Person weight
- **COUNT**: A column used to create the counts column (all entries are 1).


As stated above, `dep_lst` and `option_col` need to specified as one/more of the field descriptions in the PUMA data csv file as follows. The table below provides a mapping of some of the PUMA data csc columns to the present housing characteristic columns available. 

| PUMA description | Present housing xtic. description |
|------------------|-----------------------------------|
| UNITSSTR         | Geometry Building Type ACS        |
| GBT_RECS**       | Geometry Building Type RECS       |
| BEDROOMS         | Bedrooms                          |
| BUILTYR2         | Vintage                           |
| BUILTYR2_ACS**   | Vintage ACS                       |
| FUELHEAT         | Heating Fuel                      |

*This column does not exist presently in the PUMA data csv file. However, `dep_lst` or `option_col` can be specifed as "BUILTYR2_ACS" or "GBT_RECS".*

# Import modules

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from IPython.display import display

## Initialize Object

In [2]:
# Try to get the tsv_maker if it exists, then reload
try:
    del sys.modules['tsv_maker_pums']
except KeyError:
    pass
from tsv_maker_pums import TSVMaker # Class methods to create the spatial tsvs

# Inputs
write_tf = True
project= os.path.join('..','..','project_multifamily_beta','housing_characteristics') # select project folder

# Initialize PUMA tsv_maker object
puma_tsv = TSVMaker(project)

print(np.shape(puma_tsv.puma_df))
print(puma_tsv.puma_df.columns.values)
display(puma_tsv.puma_df.tail(10))

Initializing PUMA TSVMaker
---------------------
PUMA Files...
Loading PUMA data csv file - this may take a while
Create longPUMA column...
Loading PUMS data complete.
(14919402, 31)
['YEAR' 'MULTYEAR' 'SAMPLE' 'SERIAL' 'CBSERIAL' 'HHWT' 'STATEICP'
 'STATEFIP' 'COUNTYICP' 'COUNTYFIP' 'PUMA' 'GQ' 'OWNERSHP' 'OWNERSHPD'
 'COSTELEC' 'COSTGAS' 'COSTWATR' 'COSTFUEL' 'VACANCY' 'ROOMS' 'BUILTYR2'
 'UNITSSTR' 'BEDROOMS' 'FUELHEAT' 'PERNUM' 'PERWT' 'StateAbbrev'
 'longPUMA' 'BUILTYR2_ACS' 'GBT_RECS' 'COUNT']


,YEAR,MULTYEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,STATEICP,STATEFIP,COUNTYICP,COUNTYFIP,...,UNITSSTR,BEDROOMS,FUELHEAT,PERNUM,PERWT,StateAbbrev,longPUMA,BUILTYR2_ACS,GBT_RECS,COUNT
14919392,2017.0,2017.0,201703.0,6872746.0,2.017001e+12,24.0,68.0,56.0,0.0,0.0,...,9.0,3.0,2.0,2.0,26.0,WY,"WY, 00400",6.0,9.0,1
14919393,2017.0,2017.0,201703.0,6872747.0,2.017001e+12,16.0,68.0,56.0,0.0,0.0,...,3.0,4.0,3.0,1.0,16.0,WY,"WY, 00200",9.0,3.0,1
14919394,2017.0,2017.0,201703.0,6872747.0,2.017001e+12,16.0,68.0,56.0,0.0,0.0,...,3.0,4.0,3.0,2.0,12.0,WY,"WY, 00200",9.0,3.0,1
14919395,2017.0,2017.0,201703.0,6872748.0,2.017001e+12,26.0,68.0,56.0,0.0,0.0,...,3.0,5.0,2.0,1.0,26.0,WY,"WY, 00400",4.0,3.0,1
14919396,2017.0,2017.0,201703.0,6872748.0,2.017001e+12,26.0,68.0,56.0,0.0,0.0,...,3.0,5.0,2.0,2.0,13.0,WY,"WY, 00400",4.0,3.0,1
14919397,2017.0,2017.0,201703.0,6872749.0,2.017001e+12,26.0,68.0,56.0,0.0,0.0,...,7.0,1.0,4.0,1.0,26.0,WY,"WY, 00300",7.0,7.0,1
14919398,2017.0,2017.0,201703.0,6872750.0,2.017001e+12,15.0,68.0,56.0,0.0,0.0,...,7.0,2.0,4.0,1.0,15.0,WY,"WY, 00100",7.0,7.0,1
14919399,2017.0,2017.0,201703.0,6872751.0,2.017001e+12,9.0,68.0,56.0,0.0,0.0,...,3.0,4.0,2.0,1.0,9.0,WY,"WY, 00400",5.0,3.0,1
14919400,2017.0,2017.0,201703.0,6872752.0,2.017001e+12,13.0,68.0,56.0,0.0,0.0,...,3.0,4.0,3.0,1.0,12.0,WY,"WY, 00300",6.0,3.0,1
14919401,2017.0,2017.0,201703.0,6872753.0,2.017001e+12,8.0,68.0,56.0,0.0,0.0,...,9.0,3.0,4.0,1.0,8.0,WY,"WY, 00400",7.0,9.0,1


# Write reformatted PUMA tsv to project folder

The table below provides the inputs that would be need to create some of the common housing characteristics tsvs:

| TSV File                   | dep_lst                                  | option_col     |
|----------------------------|------------------------------------------|----------------|
| Geometry Building File FPL | ["longPUMA"]                             | "UNITSSTR"     |
| Vintage FPL                | ["longPUMA","UNITSSTR"]                  | "BUILTYR2_FPL" |
| Vintage                    | ["longPUMA", "BUILTYR2_ACS"]             | "BUILTYR2"     |
| Heating Fuel               | ["longPUMA", "UNITSSTR", "BUILTYR2_ACS"] | "FUELHEAT"     |

## Create Geometry Building Type ACS

In [3]:
# for Geometry Building Type FPL, for example,run the following:
dep_list = ['longPUMA']
option_col = "UNITSSTR"

puma_tsv.create_tsv_with_dependencies(dep_list,option_col)

# Write new housing characteristics to project folder
if write_tf:
    puma_tsv.write_tsv_to_projects()

Creating mapping...
Pivoting table...
Formating table into tsv...
All done! file(s) written into tsv paths!


## Create Vintage

In [4]:
# for Geometry Building Type FPL, for example,run the following:
dep_list = ["longPUMA","UNITSSTR"]
option_col = "BUILTYR2"

puma_tsv.create_tsv_with_dependencies(dep_list,option_col)

# Write new housing characteristics to project folder
if write_tf:
    puma_tsv.write_tsv_to_projects()

Creating mapping...
Pivoting table...
Formating table into tsv...
All done! file(s) written into tsv paths!


## Create Vintage ACS
Since we have the PUMS data, we can break out vintage (decadal bins). Vintage ACS then becomes a mapping TSV.

In [5]:
# for Geometry Building Type FPL, for example,run the following:
dep_list = ['BUILTYR2']
option_col = "BUILTYR2_ACS"

puma_tsv.create_tsv_with_dependencies(dep_list,option_col)

# Write new housing characteristics to project folder
if write_tf:
    puma_tsv.write_tsv_to_projects()

Creating mapping...
Pivoting table...
Formating table into tsv...
All done! file(s) written into tsv paths!


## Create Heating Fuel
The number of cross tabs is still a bit large. Some rows have 0 counts. The combination was replaced with the state average of the Geometry Building Type RECS and the Vintage ACS.

In [6]:
# for Geometry Building Type FPL, for example,run the following:
dep_list = ["longPUMA","GBT_RECS","BUILTYR2_ACS"]
option_col = "FUELHEAT"

# Create the initial dataframe from the data
puma_tsv.create_tsv_with_dependencies(dep_list,option_col)

# Get the state
puma_tsv.pivot_df['state'] = [puma.split(',')[0] for puma in puma_tsv.pivot_df['Dependency=PUMA']]

# Calculate State Average
deps = ['Dependency=Geometry Building Type RECS','Dependency=Vintage ACS','state']
option_cols = [s for s in puma_tsv.pivot_df.columns.values if "Option=" in s]
state_ave_df = puma_tsv.pivot_df.copy()
for col in option_cols:
    state_ave_df[col] = state_ave_df[col]*state_ave_df['source_weight']
state_ave_df = state_ave_df.groupby(deps).sum().reset_index(drop=False)
state_ave_df[option_cols] = state_ave_df[option_cols].div(state_ave_df[option_cols].sum(axis=1),axis=0)
state_ave_df.set_index(deps, inplace=True)

# Fill 0 count rows with state weighted average
option_cols.append('source_count')
option_cols.append('source_weight')
idx = np.where(puma_tsv.pivot_df['source_count'] == 0)[0]
for i in idx:
    jdx = tuple(puma_tsv.pivot_df[deps].loc[i])
    row = state_ave_df.xs(jdx)
    puma_tsv.pivot_df.loc[i,option_cols] = row

# Remove state column    
puma_tsv.pivot_df = puma_tsv.pivot_df[puma_tsv.pivot_df.columns.values[:-1]]

# Write new housing characteristics to project folder
if write_tf:
    puma_tsv.write_tsv_to_projects()

Creating mapping...
Pivoting table...
Formating table into tsv...
All done! file(s) written into tsv paths!
